In [ ]:
# notebook used for transforming imaging reports in JSON format into CSV file

In [1]:
import numpy as np
import pandas as pd
import json
import glob

In [2]:
prefix = '/mnt/d/Dropbox (Partners Healthcare)/profile_3-2023/'


In [7]:
def parse_imaging_json(file_name):
    with open(file_name, encoding='utf-8') as json_file:
        imaging_dict = json.load(json_file)
    imaging_frame = pd.DataFrame.from_records(imaging_dict['response']['docs'])   
    imaging_frame = imaging_frame[['DFCI_MRN','EVENT_DATE','RPT_TEXT','IMPRESSION_TEXT','PROC_DESC']]
    imaging_frame = imaging_frame.rename(columns={'DFCI_MRN':'dfci_mrn', 'EVENT_DATE':'date', 'RPT_TEXT':'text', 'IMPRESSION_TEXT':'impression_text','PROC_DESC':'scan_type'})
    
    imaging_frame.date = pd.to_datetime(imaging_frame.date.str[:10])
    
    imaging_frame.text = imaging_frame.text.str.replace('\n|\r', ' ')
    imaging_frame.text = imaging_frame.text.str.encode('ascii', 'ignore')
    imaging_frame.text = [str(x, 'ascii') for x in imaging_frame.text]
    imaging_frame.text = imaging_frame.text.str.replace('20\d\d',' ')

    imaging_frame['text'] = imaging_frame.text.str.replace("\\s+", " ", regex=True)


    #imaging_frame.text = imaging_frame.text.str.lower()
    
    imaging_frame = imaging_frame[~imaging_frame.scan_type.str.contains('XR ')]
    
    imaging_frame = imaging_frame[~imaging_frame.scan_type.str.contains('PACS')]

    
    return imaging_frame
        

In [8]:
filename_list = [x for x in glob.glob(prefix+'/text/imaging/*Imag*.json')]
filename_list

['d:/Dropbox (Partners Healthcare)/profile_3-2023//text/imaging\\RequestID-105304-0323-Imaging-1.json',
 'd:/Dropbox (Partners Healthcare)/profile_3-2023//text/imaging\\RequestID-105304-0323-Imaging-2.json',
 'd:/Dropbox (Partners Healthcare)/profile_3-2023//text/imaging\\RequestID-105304-0323-Imaging-3.json',
 'd:/Dropbox (Partners Healthcare)/profile_3-2023//text/imaging\\RequestID-105304-0323-Imaging-4.json']

In [9]:
report_list = [parse_imaging_json(x) for x in filename_list]

C:\Users\kk71\AppData\Local\Temp\ipykernel_16428\4205139122.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  imaging_frame.text = imaging_frame.text.str.replace('\n|\r', ' ')
C:\Users\kk71\AppData\Local\Temp\ipykernel_16428\4205139122.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  imaging_frame.text = imaging_frame.text.str.replace('20\d\d',' ')
C:\Users\kk71\AppData\Local\Temp\ipykernel_16428\4205139122.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  imaging_frame.text = imaging_frame.text.str.replace('\n|\r', ' ')
C:\Users\kk71\AppData\Local\Temp\ipykernel_16428\4205139122.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  imaging_frame.text = imaging_frame.text.str.replace('20\d\d',' ')
C:\Users\kk71\AppData\Local\Temp\ipykernel_16428\4205139122.py:11: FutureWarning: Th

In [13]:
report_frame = pd.concat(report_list, axis=0)

In [14]:
report_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1327678 entries, 0 to 254106
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   dfci_mrn         1327678 non-null  object        
 1   date             1327678 non-null  datetime64[ns]
 2   text             1327678 non-null  object        
 3   impression_text  635159 non-null   object        
 4   scan_type        1327678 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 60.8+ MB


In [16]:
registration = pd.read_csv(prefix+'structured_data/REQ_KK71_105304_2_PT_INFO_STATUS_REGISTRATION.csv', sep=',', encoding='iso-8859-1', low_memory=False)[['DFCI_MRN', 'PATIENT_ID', 'HYBRID_DEATH_IND', 'HYBRID_DEATH_DT']].rename(columns={'DFCI_MRN':'dfci_mrn', 'PATIENT_ID':'patient_id', 'HYBRID_DEATH_IND':'hybrid_death_ind', 'HYBRID_DEATH_DT':'hybrid_death_dt'})
registration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56870 entries, 0 to 56869
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dfci_mrn          56870 non-null  int64 
 1   patient_id        56870 non-null  int64 
 2   hybrid_death_ind  56870 non-null  object
 3   hybrid_death_dt   22955 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.7+ MB


In [17]:
report_frame['dfci_mrn'] = pd.to_numeric(report_frame.dfci_mrn)
report_frame = pd.merge(report_frame, registration, on='dfci_mrn')


In [22]:
# pull tumor sequencing data to get cancer diagnosis per Oncotree and NGS date
specimens = pd.read_csv(prefix+'structured_data/REQ_KK71_105304_2_GENOMIC_SPECIMEN.csv', encoding='ISO-8859-1', low_memory=False)
specimens = specimens[['PATIENT_ID','PRIMARY_CANCER_DIAGNOSIS', 'TEST_ORDER_DT']].groupby('PATIENT_ID').first().reset_index().rename(columns={'PATIENT_ID':'patient_id', 'PRIMARY_CANCER_DIAGNOSIS':'primary_cancer_diagnosis', 'TEST_ORDER_DT':'genomics_date'})
specimens = pd.merge(specimens, registration, on='patient_id')
specimens = specimens[~specimens.primary_cancer_diagnosis.isnull()]
# no leukemia or myeloma
specimens = specimens[~specimens.primary_cancer_diagnosis.str.contains("Leukemia|Myeloma")]
specimens['genomics_date'] = pd.to_datetime(specimens.genomics_date)
specimens = specimens.sort_values(by=['dfci_mrn','genomics_date']).reset_index(drop=True)
specimens = specimens.groupby('dfci_mrn').first().reset_index()
specimens = specimens[['dfci_mrn','primary_cancer_diagnosis', 'genomics_date']]
specimens.primary_cancer_diagnosis.value_counts()

Lung Adenocarcinoma                    3607
Colon Adenocarcinoma                   2477
Breast Invasive Ductal Carcinoma       1550
Pancreatic Adenocarcinoma              1504
Glioblastoma                           1366
                                       ... 
Gestational Trophoblastic Disease         1
Complete Hydatidiform Mole                1
Metaplastic Squamous Cell Carcinoma       1
Liver Angiosarcoma                        1
Renal Medullary Carcinoma                 1
Name: primary_cancer_diagnosis, Length: 474, dtype: int64

In [23]:
specimens.shape[0]

35287

In [ ]:
report_frame = pd.merge(report_frame, specimens, on=['dfci_mrn'])
report_frame['after_profile'] = np.where(report_frame.genomics_date.isnull() | (report_frame.date < report_frame.genomics_date), 0, 1)
report_frame['hybrid_death_dt'] = pd.to_datetime(report_frame.hybrid_death_dt)


In [25]:
# pull train/val/test split
split = pd.read_csv(prefix+'derived_data/split_3-2023.csv')

In [26]:
final_output = pd.merge(report_frame, split[['dfci_mrn', 'split']], on='dfci_mrn')
# no external scans without interpretation
final_output = final_output[~final_output.scan_type.str.contains('NO INTERPRETATION')]
final_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 776304 entries, 2 to 946820
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   dfci_mrn                  776304 non-null  int64         
 1   date                      776304 non-null  datetime64[ns]
 2   text                      776304 non-null  object        
 3   impression_text           492240 non-null  object        
 4   scan_type                 776304 non-null  object        
 5   patient_id                776304 non-null  int64         
 6   hybrid_death_ind          776304 non-null  object        
 7   hybrid_death_dt           398506 non-null  datetime64[ns]
 8   primary_cancer_diagnosis  776304 non-null  object        
 9   genomics_date             776304 non-null  datetime64[ns]
 10  after_profile             776304 non-null  int32         
 11  split                     776304 non-null  object        
dtypes:

In [27]:
prefix

'd:/Dropbox (Partners Healthcare)/profile_3-2023/'

In [28]:
final_output.to_csv(prefix+'derived_data/all_imaging_for_prissmm.csv')

In [29]:
final_output = pd.read_csv(prefix+'derived_data/all_imaging_for_prissmm.csv')

In [30]:
final_output.scan_type.value_counts()[0:20]

CT CHEST                                105559
CT ABDOMEN/PELVIS                        91581
MRI BRAIN                                47445
CT CHEST WITH CONTRAST                   41924
HISTORICAL RADIOLOGY RESULT, NOS         34194
MRI BRAIN WITH AND WITHOUT CONTRAST      22886
NM PET CT SKULL BASE TO MID THIGHS       21305
CT ABDOMEN/PELVIS (GI) WITH CONTRAST     17450
CT HEAD                                  13342
CT ABDOMEN/PELVIS WITH CONTRAST          11422
CT CHEST WITHOUT CONTRAST                10814
BI MAMMOGRAM SCREENING (BILATERAL)       10484
MRI ABDOMEN                              10208
CT NECK                                   8353
CT CHEST PULMONARY ANGIOGRAM              7964
NM BONE SCAN WHOLE BODY                   7913
NM PET CT                                 7593
BI MAMMOGRAM SCREENING                    7396
MRI LUMBAR SPINE                          6443
CT HEAD WITHOUT CONTRAST                  6136
Name: scan_type, dtype: int64